In [ ]:
#unzip NEU dataset
!unzip '/content/drive/MyDrive/NEU/100_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/NEU_validation.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/NEU_test.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/50_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/20_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/10_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/5_samples.zip' -d '/content'
!unzip '/content/drive/MyDrive/NEU/3_samples.zip' -d '/content'


"\n!unzip '/content/drive/MyDrive/NEU/100_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/NEU_validation.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/NEU_test.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/50_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/20_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/10_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/5_samples.zip' -d '/content'\n!unzip '/content/drive/MyDrive/NEU/3_samples.zip' -d '/content'\n"

In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
from sklearn.metrics import classification_report
import imgaug.augmenters as iaa
import imgaug as ia
import os

In [ ]:
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
#load NEU dataset
def load_data(dir_path, labels_dict):
  all_names = os.listdir(dir_path)
  data_x = []
  data_y = []
  for name in all_names:
    img = keras.preprocessing.image.load_img(os.path.join(dir_path, name))
    img = keras.preprocessing.image.img_to_array(img,dtype='uint8')
    data_x.append(img)
    lab = labels_dict[name[:2]]
    data_y.append(lab)
  data_x = np.array(data_x)
  data_y = np.array(data_y)
  return data_x, data_y
class NEUdata(keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size, train):
      self.data_x = data_x
      self.data_y = data_y
      self.train = train
      self.batch_size = batch_size

    def __len__(self):
      return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):
      batch_x = self.data_x[idx * self.batch_size:(idx + 1) * self.batch_size]
                            
      batch_y = self.data_y[idx * self.batch_size:(idx + 1) * self.batch_size]
      if self.train == True:
          batch_x = Data_Aug(batch_x)

      batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
      batch_y = keras.utils.to_categorical(batch_y, 6)
      return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#save the best model during training 
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        #self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') 
                #+ logs.get(
                 #   'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#Cosine&Euclidean distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=3)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X=tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  


In [ ]:
#create single AlexNet model
def create_AlexNet(input_shape, num_classes=6, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(96, (3,3), strides=(2,2), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x) 
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  feature_maps = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Flatten()(feature_maps)
  #x = keras.layers.Dense(256,activation='relu')(x)
  #x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(128,activation='relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,feature_maps])
  return model

In [ ]:
#load training, validation and testing images
training_data_path = '3_samples'
validation_data_path = 'NEU_validation'
test_data_path = 'NEU_test'
labels_dict = {'Cr': 0, 'In': 1, 'Pa': 2, 'PS': 3, 'RS': 4, 'Sc': 5}
x_train, y_train = load_data(training_data_path, labels_dict)
x_val, y_val = load_data(validation_data_path, labels_dict)
x_test, y_test = load_data(test_data_path, labels_dict)
x_test = x_test/255.0

In [ ]:
#construct joint training for the five base models
input_size = (200, 200, 3)
model0 = create_AlexNet(input_shape=input_size, num_classes=6,  random_seed=1)
model1 = create_AlexNet(input_shape=input_size, num_classes=6,  random_seed=2)
model2 = create_AlexNet(input_shape=input_size, num_classes=6,  random_seed=3)
model3 = create_AlexNet(input_shape=input_size, num_classes=6,  random_seed=4)
model4 = create_AlexNet(input_shape=input_size, num_classes=6,  random_seed=5)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(
    inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=1, euclidean_weight=10, mask=True, max_norm=False)

model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')

model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])

#construct training and validation samples
train_sequence = NEUdata(x_train, y_train, batch_size=10, train=True)
validation_sequence = NEUdata(x_val, y_val, batch_size=10, train=False)

#training 
history = model_train.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
#frame.to_excel('table_{numsample}_samples_seed_{numseed}.xlsx'.format(numsample=100, numseed='fix'))


Epoch 1/300
2/2 [==============================] - 19s 6s/step - loss: 41.9142 - model_loss: 8.2094 - model_1_loss: 7.3877 - model_2_loss: 6.8176 - model_3_loss: 11.2966 - model_4_loss: 8.2029 - model_accuracy: 0.1111 - model_1_accuracy: 0.1111 - model_2_accuracy: 0.1111 - model_3_accuracy: 0.0000e+00 - model_4_accuracy: 0.1667 - loss1: 9.1927 - loss2: 9.3323 - val_loss: 13.5231 - val_model_loss: 3.7230 - val_model_1_loss: 2.7450 - val_model_2_loss: 2.2833 - val_model_3_loss: 1.9857 - val_model_4_loss: 2.7861 - val_model_accuracy: 0.1583 - val_model_1_accuracy: 0.1000 - val_model_2_accuracy: 0.1667 - val_model_3_accuracy: 0.1750 - val_model_4_accuracy: 0.1667 - val_loss1: 9.2729 - val_loss2: 9.8447
Epoch 2/300
2/2 [==============================] - 1s 1s/step - loss: 25.3099 - model_loss: 5.9668 - model_1_loss: 4.3627 - model_2_loss: 4.5007 - model_3_loss: 4.2315 - model_4_loss: 6.2482 - model_accuracy: 0.2222 - model_1_accuracy: 0.3333 - model_2_accuracy: 0.2222 - model_3_accuracy: 0.

In [ ]:
#evaluation of the five base models
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate

36/36 [==============================] - 2s 28ms/step - loss: 12.2758 - model_loss: 2.9699 - model_1_loss: 2.3723 - model_2_loss: 2.4696 - model_3_loss: 1.8116 - model_4_loss: 2.6524 - model_accuracy: 0.3667 - model_1_accuracy: 0.4306 - model_2_accuracy: 0.3889 - model_3_accuracy: 0.4056 - model_4_accuracy: 0.3833 - loss1: 7.5797 - loss2: 9.7577


In [ ]:
#hard voting ensemble
class HardVotingEnsemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
HEnsemble = HardVotingEnsemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
HEnsemble.evaluate()

0.41944444444444445

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.425

In [ ]:
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        #self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        #self.model.save(self.save_path)

In [ ]:
class eNEUdata(keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size, train):
      self.data_x = data_x
      self.data_y = data_y
      self.train = train
      self.batch_size = batch_size

    def __len__(self):
      return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):
      batch_x = self.data_x[idx * self.batch_size:(idx + 1) * self.batch_size]
                            
      batch_y = self.data_y[idx * self.batch_size:(idx + 1) * self.batch_size]
      if self.train == True:
          batch_x = Data_Aug(batch_x)

      batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
      batch_y = keras.utils.to_categorical(batch_y, 6)
      return batch_x, batch_y

In [ ]:
#feature fusion model
input_size = (200, 200, 3)
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[12].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[12].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[12].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[12].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[12].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)

x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(6, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = eNEUdata(x_train, y_train, batch_size=10, train=True)
validation_sequence = eNEUdata(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test))

Epoch 1/50
2/2 [==============================] - 4s 1s/step - loss: 2.0433 - accuracy: 0.3889 - val_loss: 18.3346 - val_accuracy: 0.3833
Epoch 2/50
2/2 [==============================] - 1s 920ms/step - loss: 1.2985 - accuracy: 0.5556 - val_loss: 10.5596 - val_accuracy: 0.3806
Epoch 3/50
2/2 [==============================] - 1s 1s/step - loss: 1.0808 - accuracy: 0.7222 - val_loss: 7.3208 - val_accuracy: 0.3972
Epoch 4/50
2/2 [==============================] - 1s 1s/step - loss: 0.9328 - accuracy: 0.6667 - val_loss: 6.0198 - val_accuracy: 0.4167
Epoch 5/50
2/2 [==============================] - 1s 914ms/step - loss: 0.7018 - accuracy: 0.8889 - val_loss: 5.1627 - val_accuracy: 0.4139
Epoch 6/50
2/2 [==============================] - 1s 912ms/step - loss: 1.2560 - accuracy: 0.5556 - val_loss: 4.5152 - val_accuracy: 0.4167
Epoch 7/50
2/2 [==============================] - 1s 916ms/step - loss: 0.6410 - accuracy: 0.8889 - val_loss: 4.1926 - val_accuracy: 0.4139
Epoch 8/50
2/2 [===========

In [ ]:
#finetuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
#train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
#validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test))

Epoch 1/20
2/2 [==============================] - 4s 1s/step - loss: 0.4555 - accuracy: 0.8333 - val_loss: 2.0918 - val_accuracy: 0.4556
Epoch 2/20
2/2 [==============================] - 1s 1s/step - loss: 0.4308 - accuracy: 0.8889 - val_loss: 2.0955 - val_accuracy: 0.4583
Epoch 3/20
2/2 [==============================] - 1s 1s/step - loss: 0.4100 - accuracy: 0.8889 - val_loss: 2.0861 - val_accuracy: 0.4639
Epoch 4/20
2/2 [==============================] - 1s 946ms/step - loss: 0.3127 - accuracy: 0.9444 - val_loss: 2.0849 - val_accuracy: 0.4611
Epoch 5/20
2/2 [==============================] - 1s 1s/step - loss: 0.6028 - accuracy: 0.8333 - val_loss: 2.0715 - val_accuracy: 0.4667
Epoch 6/20
2/2 [==============================] - 1s 964ms/step - loss: 0.4003 - accuracy: 0.8889 - val_loss: 2.0572 - val_accuracy: 0.4667
Epoch 7/20
2/2 [==============================] - 1s 1s/step - loss: 0.3947 - accuracy: 0.8889 - val_loss: 2.0486 - val_accuracy: 0.4694
Epoch 8/20
2/2 [===================

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.5978    0.9167    0.7237        60\n           1     0.4038    0.3500    0.3750        60\n           2     0.9231    0.8000    0.8571        60\n           3     0.2500    0.0333    0.0588        60\n           4     0.0000    0.0000    0.0000        60\n           5     0.2869    0.5833    0.3846        60\n\n    accuracy                         0.4472       360\n   macro avg     0.4103    0.4472    0.3999       360\nweighted avg     0.4103    0.4472    0.3999       360\n'